# 02467 Assignment 2
s204085 Cornelius Erichs & s204076 Natasha Hougaard

#### GitHub repository
Here is our repository on GitHub: https://github.com/natasha0301/Assigment2-CSS


## Part 1: Mixing Patterns and Assortativity

For each node, compute the fraction of edges that connect to a node that works in the same top field. Find the average value across all nodes.

In [1]:
from networkx.readwrite import json_graph
import networkx as nx
import json
import numpy as np
import pandas as pd

def read_json_file(filename):
    with open(filename) as f:
        js_graph = json.load(f)
    return json_graph.node_link_graph(js_graph)
G = read_json_file("data_total.json")

def same_field(G):
    same_field_fractions = []
    for node in G.nodes():
        same_field_neighbors = 0
        total_neighbors = 0
        
        for neighbor in G.neighbors(node):
            if G.nodes[neighbor]["field"] == G.nodes[node]["field"]:
                same_field_neighbors += 1
            total_neighbors += 1
        
        if total_neighbors > 0:
            same_field_fraction = same_field_neighbors / total_neighbors
        else:
            same_field_fraction = 0
        same_field_fractions.append(same_field_fraction)
    return same_field_fractions

same_field_fractions = same_field(G)
print(np.mean(same_field(G)))

0.5842856084183233


Create a new graph, with the same nodes and edges, but where the association between nodes and field is shuffled. Compute the measure above for this randomized graph.

In [36]:
import random

shuffled_G = G.copy()

fields = [G.nodes[node]["field"] for node in G.nodes()]
random.shuffle(fields)

for i, node in enumerate(shuffled_G.nodes()):
    shuffled_G.nodes[node]["field"] = fields[i]
    
print(np.mean(same_field(shuffled_G)))

0.1787791531350391


Repeat the point above 100 times (at least). Plot the distribution of the values obtained and compare it with the value you have found for the real graph. Is the chance to connect to a member of the same field significantly higher than it would be by chance?

In [37]:
import matplotlib.pyplot as plt
results = []
for i in range(500):
    random.shuffle(fields)

    for i, node in enumerate(shuffled_G.nodes()):
        shuffled_G.nodes[node]["field"] = fields[i]
    
    results.append(np.mean(same_field(shuffled_G)))
    
plt.hist(results)

Compute the assortativity coefficient with respect to author's field. How do you interpret the value you obtain? 

In [ ]:
print(np.unique(fields))
unique_fields = {'Agricultural And Food Sciences' : 0,
            'Art' : 1,
            'Business' : 2,
            'Chemistry' : 3,
            'Computer Science' : 4,
            'Economics' : 5,
            'Education' : 6,
            'Engineering' : 7,
            'Environmental Science' : 8,
            'Geography' : 9,
            'Geology' : 10,
            'History' : 11,
            'Law' : 12,
            'Linguistics': 13,
            'Materials Science': 14,
            'Mathematics': 15,
            'Medicine': 16,
            'Philosophy' : 17,
            'Physics' : 18,
            'Political Science' : 19,
            'Psychology' : 20,
            'Sociology' : 21,
            None:22}

matrix = np.zeros((len(unique_fields),len(unique_fields)))

values = nx.get_node_attributes(G, "field").values()
num_values = len(values)

for start, end in G.edges(): # Looping over all edges in graph (since its undirected its not really start and stop)
    x = G.nodes[start]["field"] # Getting the start point of the edge 
    y = G.nodes[end]["field"] # Getting the end point of the edge 
    if x in unique_fields: 
        x = unique_fields[x]
    else:
        x = unique_fields[None] # in case x is nan
        
    if y in unique_fields:
        y = unique_fields[y]
    else:
        y = unique_fields[None] # in case y is nan
    matrix[x, y] += 1 
    
num_edges = len(G.edges())
matrix /= num_edges # averaging the occurence with the total edges

trace = np.trace(matrix) # trace of the matrix, the sum of the diagonal entries
mix_matrix = np.sum(np.matmul(matrix, matrix))

r1 = (trace-mix_matrix)/(1-mix_matrix) # Eq. 2
print(r1)

['Agricultural And Food Sciences' 'Art' 'Business' 'Chemistry'
 'Computer Science' 'Economics' 'Education' 'Engineering'
 'Environmental Science' 'Geography' 'Geology' 'History' 'Law'
 'Linguistics' 'Materials Science' 'Mathematics' 'Medicine' 'Philosophy'
 'Physics' 'Political Science' 'Psychology' 'Sociology' 'nan']
0.7106920785154831


The result of 0.71 indicated some clusters of nodes are formed based on their top field, but there are also occurences where this is not the case. With 0 indicating random mixing we can see that their field plays some kind of role.

Is the graph assortative with respect to the degree? (e.g. do high-degree scientists tend to link to other high-degree scientists, and low-degree scientists to other low-degree scientists?). Provide an interpretation of your answer.

If the assortativity coefficient is > 0, it suggests that nodes with similar degrees tend to be connected to each other, indicating assortative mixing w.r.t. the degree. And if the assortativity coefficient is < 0, it suggests that nodes with different degrees tend to be connected to each other, indicating disassortative mixing w.r.t. the degree. 

A value of 0.71 for the assortativity coefficient suggests that there is a degree of assortative mixing for the authors with respect to the their top fields. In other words, authors with the same top field are more likely to be connected to each other than to nodes with different top fields.

## Part 2: Communities

**EXERCISE 2: ZACHARYS'S KARATE CLUB**

In this exercise, we will work on Zarachy's karate club graph (refer to the Introduction of Chapter 9). The dataset is available in NetworkX, by calling the function karate_club_graph

1. Visualize the graph using netwulf. Set the color of each node based on the club split

In [2]:
import matplotlib.pyplot as plt
import networkx as nx
from netwulf import visualize

G = nx.karate_club_graph()

#First we set the color of each node based on the club split
colors = ['red' if G.nodes[n]['club'] == 'Mr. Hi' else 'blue' for n in G.nodes()]

#Then we add the colors to the node attributes
for i, n in enumerate(G.nodes()):
    G.nodes[n]['color'] = colors[i]

#Visualize the graph
visualize(G)

(None, None)

2. Write a function to compute the modularity of a graph partitioning using equation 9.12 in the book: 

$M=\sum_{c=1}^{n_c}\left[\frac{L_c}{L}-(\frac{k_c}{2L})^2\right]$ 



The function should take a networkX Graph and a partitioning as inputs and return the modularity.

In [ ]:
"""
def compute_modularity(G,partitioning):
    L = G.number_of_edges()
    M = 0
    communities = set(partitioning.values())

    for community in communities:
        nodes_in_community = [node for node, community_id in partitioning.items() if community_id == community]
        subgraph = G.subgraph(nodes_in_community)
        k_c = sum(dict(subgraph.degree()).values())
        L_c = subgraph.number_of_edges()
        M += L_c / L - (k_c / (2 * L)) ** 2

    return M
"""

'\ndef compute_modularity(G,partitioning):\n    L = G.number_of_edges()\n    M = 0\n    communities = set(partitioning.values())\n\n    for community in communities:\n        nodes_in_community = [node for node, community_id in partitioning.items() if community_id == community]\n        subgraph = G.subgraph(nodes_in_community)\n        k_c = sum(dict(subgraph.degree()).values())\n        L_c = subgraph.number_of_edges()\n        M += L_c / L - (k_c / (2 * L)) ** 2\n\n    return M\n'

3. Explain in your own words the concept of modularity **REFRASE**

Modularity is a measure of the structure of a network, which is a collection of nodes (also known as vertices) and the edges (also known as links) that connect them. The concept of modularity is based on the idea that a well-connected network can be divided into groups of nodes, known as communities or clusters, where the nodes within each group are more densely connected to each other than to nodes outside the group.

Modularity is a measure of how well a network can be divided into such communities. Specifically, modularity quantifies the difference between the number of edges within communities and the expected number of edges within communities if the edges were placed randomly, while preserving the degree distribution of the nodes. In other words, modularity measures the extent to which the edges in a network are concentrated within communities, rather than being distributed randomly throughout the network.

Modularity is typically expressed as a number between -1 and 1. A high positive value of modularity indicates that the network is highly modular, with many densely connected communities, while a low or negative value indicates that the network is less modular, with communities that are not well-defined. The concept of modularity has many applications in network science, including community detection, network visualization, and network comparison.

4. Compute the modularity of the Karate club split partitioning using the function you just wrote

In [3]:
import networkx as nx

def compute_modularity(G,partitioning):
    L = G.number_of_edges()
    M = 0
    communities = set(partitioning.values())

    for community in communities:
        nodes_in_community = [node for node, community_id in partitioning.items() if community_id == community]
        subgraph = G.subgraph(nodes_in_community)
        k_c = sum(dict(subgraph.degree()).values())
        L_c = subgraph.number_of_edges()
        M += L_c / L - (k_c / (2 * L)) ** 2

    return M

#We start by loading the Karate club graph
G = nx.karate_club_graph()

#First we get the club split partitioning from the node attributes
club_split = nx.get_node_attributes(G, "club")

#Then we compute the modularity of the partitioning by using the function above
modularity = compute_modularity(G, club_split)

print(f"The modularity of the Karate club split partitioning using the function is {modularity:.3f}")

The modularity of the Karate club split partitioning using the function is 0.489


5. We will now perform a small randomization experiment to assess if the modularity you just computed is statitically different from 0. To do so, we will implement the double edge swap algorithm. Given a network G, this algorithm creates a new network, such that each node has exactly the same degree as in the original network, but different connections. Here is how the algorithm works.

    a. Create an identical copy of your original network.
    
    b. Consider two edges in your new network (u,v) and (x,y), such that u!=v and v!=x.
    
    c. If none of edges (u,y) and (x,v) exists already, add them to the network and remove edges (u,v) and (x,y).
    
Repeat steps b. and c. to achieve at least N swaps (I suggest N to be larger than the number of edges).

#### Exercise 3: 
Community detection on the network of Computational Social Scientists.



Use the Python Louvain-algorithm implementation to find communities. How many communities do you find? What are their sizes? Report the value of modularity found by the algorithm. Is the modularity significantly different than 0?


In [1]:
from networkx.readwrite import json_graph
import networkx as nx
import json
import numpy as np
import pandas as pd
from netwulf import visualize

def read_json_file(filename):
    with open(filename) as f:
        js_graph = json.load(f)
    return json_graph.node_link_graph(js_graph)
G = read_json_file("data_total.json")

In [2]:
import community

#Louvain algorithm
partition = community.best_partition(G) # dictionary, keys are the nodes and values are communities for each node

# modularity of partition
modularity = community.modularity(partition, G) 

size = []
for community_ in set(partition.values()):
    temp = []
    for part in partition:
        if partition[part] == community_:
            temp.append(part)
    size.append(len(temp))

print("Number of communities:", len(set(partition.values())))
print("Community sizes:", size)
print("Modularity:", modularity)

expected_modularity = community.modularity(partition, G, weight=None) # expected modularity of a null model with the same degree
if modularity > expected_modularity:
    print("modularity significantly different than 0")
else:
    print("modularity not different than 0")

Number of communities: 254
Community sizes: [3, 195, 87, 116, 5, 2, 2, 2, 259, 111, 2913, 2, 3, 2, 3, 2, 2, 2, 33, 12, 2024, 357, 2, 1130, 2, 5, 217, 210, 10, 14, 330, 2, 2, 2, 193, 4, 207, 3, 2, 237, 2, 2, 21, 2, 105, 3, 18, 2, 157, 137, 2, 2, 2, 5, 2, 477, 4, 776, 3, 4, 74, 3, 160, 3, 2, 2, 146, 2, 2, 2, 4, 3, 2, 2, 2, 2, 4, 2, 4, 4, 2, 3, 3, 4, 2, 27, 2, 27, 40, 3, 2, 2, 7, 4, 5, 3, 6, 2, 524, 228, 2, 2, 2, 3, 2, 2, 25, 3, 2, 254, 3, 2, 2, 6, 2, 2, 2, 2, 4, 3, 2, 3, 3, 3, 5, 3, 2, 160, 3, 2, 10, 2, 2, 2, 5, 3, 4, 2, 3, 13, 2, 7, 2, 2, 4, 2, 3, 2, 2, 2, 3, 121, 3, 12, 2, 8, 5, 4, 3, 30, 2, 3, 4, 2, 2, 2, 2, 7, 4, 6, 3, 3, 2, 3, 4, 2, 2, 2, 2, 2, 7, 3, 4, 4, 5, 2, 9, 3, 2, 10, 3, 2, 2, 2, 3, 2, 3, 2, 2, 2, 2, 3, 29, 6, 2, 6, 9, 5, 5, 7, 12, 11, 2, 3, 5, 12, 2, 5, 3, 3, 6, 4, 2, 4, 7, 2, 2, 2, 3, 4, 9, 3, 11, 3, 2, 2, 6, 3, 8, 2, 2, 2, 2, 2, 6, 54, 18, 4, 2, 5, 4, 6, 8, 2]
Modularity: 0.6632768922233399
Modularity is not significantly different than 0


In [14]:
largest_cc = max(nx.connected_components(G), key=len)

G_cc = G.subgraph(largest_cc)

visualize(G_cc)


(None, None)

Saving the assigned community to the author

In [18]:
nx.set_node_attributes(G, partition, 'community')

# Test
G.nodes[1404354049]

{'citation': 4,
 'year': 1871.0,
 'papers': 8,
 'field': 'Art',
 'name': 'Catherine D’ignazio',
 'community': 20}

## Part 3: TF-IDF

**Exercise 4: TF-IDF and the Computational Social Science communities**

- What does TF stand for

TF stands for term frequency. Term frequency is metric that tells us how many instances of a given word (term) there is in a document. The way we get the term frequency is by getting the set of words of a given document, and then we count how many times each word occurs and divide that by the total number of words. So if a word has a term frequency of 0.1, it means that of all the words in the document, this particular word occurs 10% of the time


- What does IDF stand for

IDF stand for inverse document frequency. IDF is a metric for how unique a word is to a given document. If the word occurs in every document we have, it is not really worth mentioning if we were asked to describe the given document


Let us take an example of why TD-IDF is used to tell which words hold the most information about a given document

If we were to only look at TF we would in most cases come to the conclusion that the word "the" is the most descriptive of a document, because it is by far the most used word in the english language. However because it is so popular it will most likely have an IDF of 0 in any corpus consisting of english documents, which means that the TF-IDF of "the" would in any english corpus yield 0


In [2]:
import nltk # natural language processing toolkit

In [3]:
# read data file
path = r"C:\Users\otto\OneDrive - Danmarks Tekniske Universitet\DTU\Kurser\02467 Computational Social Science\Code\paper_abstract_df_tokens.parquet"
path2 = r"C:\Users\otto\OneDrive - Danmarks Tekniske Universitet\DTU\Kurser\02467 Computational Social Science\Code\author_ids_df_week6.parquet"
df_tokens = pd.read_parquet(path) # contains the abstracts of the papers, and the tokens (words) in the abstracts
df_authors = pd.read_parquet(path2) # contains author ids, which community they belong to, and the authors degree (number of edges (people the author has co-authored with))



Now, we want to find out which words are important for each community, so we're going to create several *large documents, one for each community*. Each document includes all the tokens of abstracts written by members of a given community.

- Consider a community c
- Find all the abstracts of papers written by a member of community c.
- Create a long array that stores all the abstract tokens
- Repeat for all the communities.

In [52]:
# get the first commuunity
community_0 = df_authors[df_authors["community"] == 2]

community_0

,author_id,community,degree
9,2402406,2,2
10,2229725,2,2
11,3119272,2,2


In [67]:
# A function that takes a community as input and gives the paperids of the papers written by the authors in the community
def get_paperids(community):
    paperids = []
    for author in community["author_id"]:
        ids = df_tokens[df_tokens["authorIds"].apply(lambda x: author in x)][["paperId"]]
        if len(ids) > 0:
            paperids.append(ids.values[0][0])
    return list(set(paperids))


# Get the tokens of a list of paperIds
def get_tokens(paperids):
    tokens = []
    for paperid in paperids:
        temp = df_tokens[df_tokens["paperId"] == paperid]["tokens"].values
        if len(temp) > 0:
            # join the tokens into the same list
            tokens.extend(temp[0])
    return tokens


In [71]:
# now we do this process for all communities

# groupby community
grouped = df_authors.groupby("community")

# for each community apply the two functions
communities = grouped.apply(lambda x: get_tokens(get_paperids(x)))

In [78]:
# we now have a list of lists, where each list contains the tokens of the papers written by the authors in the community

community
0      [give, thorough, analytic, characterization, l...
1                                                     []
2      [seed, noisy, information, members, realworld,...
3      [point, instantiation, simpsons, paradox, covi...
4      [paper, provides, overview, legal, framework, ...
                             ...                        
306    [present, reactive, beta, model, accounts, lev...
307    [personalization, technologies, widely, used, ...
308                                                   []
309    [abstract, temporal, nature, humans, interacti...
310                                                   []
Length: 311, dtype: object

In [ ]:
# we start by implementing the more general solution, and then we can ask for the top 5 communities etc.

In [98]:
# get top 5 communites by number of authors
top_5_index = df_authors.groupby("community").count().sort_values("author_id", ascending=False).head(5).index

# get the tokens of the top 5 communities
top_5_tokens = communities[top_5_index]

# get the term frequency (TF) of the top 5 communities
top_5_tf = [nltk.FreqDist(tokens) for tokens in top_5_tokens]




In [129]:
# we define a corpus as an index so we can vary the size of the corpus
corpus = communities[top_5_index]

# get the term frequency (TF) of the corpus
tf = [nltk.FreqDist(tokens) for tokens in corpus]

In [124]:
# as we are going to anyway, let us just calculate for every word how many communities it appears in

# let us start with getting the set of all words in the corpus
all_words = set()
for tokens in corpus:
    all_words.update(tokens)


# we can then use this to calculate the tf-idf for each word in each community
tf_idf = [{word: tf[word] * np.log(len(corpus)/sum([word in tokens for tokens in corpus])) for word in all_words} for tf in top_5_tf]


In [128]:
# get the first entry
temp = tf_idf[0]

# order the words by tf-idf
sorted(temp, key=temp.get, reverse=True)

temp["populist"]

14.484941211906902